<a href="https://colab.research.google.com/github/TuragDev/ML-Python-Codes/blob/main/crystal_system_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install scikit-optimize

In [ ]:
!pip install xgboost

In [ ]:
from google.colab import files
uploaded= files.upload()

Saving Orbital_mp_test.csv to Orbital_mp_test.csv
Saving orbital_MP_train.csv to orbital_MP_train.csv
Saving sine_mp_test.csv to sine_mp_test.csv
Saving Sine_MP_train.csv to Sine_MP_train.csv
Saving Xrd_MP_train.csv to Xrd_MP_train.csv
Saving xrd_test_mp.csv to xrd_test_mp.csv


In [ ]:
import joblib
from sklearn.model_selection import train_test_split
import numpy as np

import pandas as pd

from sklearn.model_selection import GridSearchCV, RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import train_test_split

import numpy as np

import pickle
import pandas as pd
from sklearn.preprocessing import LabelEncoder

import sys
# In[3]:


from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold
from skopt import BayesSearchCV


Xrd Descriptor

In [ ]:
df1= pd.read_csv('Xrd_MP_train.csv')

df2= pd.read_csv('xrd_test_mp.csv')
df=pd.concat([df1,df2],ignore_index=True)

In [ ]:
X=df.iloc[:,14:]
Y=df['crystal_system']

In [ ]:
from sklearn.preprocessing import StandardScaler
# create a scaler object
scaler = StandardScaler()

# fit scaler on the training data
scaler.fit(X)

# apply the scaler to both the training and test data
X_normalized = scaler.transform(X)

In [ ]:
label_encoder = LabelEncoder()

# Fit and transform label encoder on target values
y_encoded = label_encoder.fit_transform(Y)

In [ ]:
import xgboost
from hyperopt import fmin, tpe, hp

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y_encoded, test_size=0.2, random_state=108)

In [19]:
# Define the search space for hyperparameters
param_space = {
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'gamma': hp.uniform('gamma', 0, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0, 5),
    'reg_lambda': hp.uniform('reg_lambda', 0, 5),
    'learning_rate': hp.loguniform('learning_rate', 0.01, 3),
    'n_estimators': hp.quniform('n_estimators', 10, 500, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1)
}

# Define the objective function to minimize (negative accuracy)
def objective(params):
    model = XGBClassifier(
        max_depth=int(params['max_depth']),
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        learning_rate=params['learning_rate'],
        n_estimators=int(params['n_estimators']),
        min_child_weight=int(params['min_child_weight'])
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return -accuracy  # Minimize negative accuracy

# Perform Bayesian optimization
best = fmin(
    fn=objective,
    space=param_space,
    algo=tpe.suggest,
    max_evals=50  # Number of optimization iterations
)

print("Best Hyperparameters:", best)

100%|██████████| 50/50 [02:53<00:00,  3.46s/trial, best loss: -0.697986577181208]
Best Hyperparameters: {'colsample_bytree': 0.8634547846864395, 'gamma': 2.1117668659478532, 'learning_rate': 1.1519455488708774, 'max_depth': 5.0, 'min_child_weight': 7.0, 'n_estimators': 372.0, 'reg_alpha': 1.7200339776916247, 'reg_lambda': 0.9824847946143372, 'subsample': 0.9212768311856734}


In [20]:
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, cross_val_score

# Define the best hyperparameters
best_hyperparameters = {
    'n_estimators': 372,
    'learning_rate': 1.1519455488708774,
    'gamma': 2.111766865947853,
    'max_depth': 5,
    'min_child_weight': 7,
    'colsample_bytree': 0.8634547846864395,
    'subsample': 0.9212768311856734,
    'reg_alpha': 1.7200339776916247,
    'reg_lambda': 0.9824847946143372
}

# Create an XGBoost classifier with the best hyperparameters
model = XGBClassifier(**best_hyperparameters)

# Perform k-fold cross-validation
cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=1)
scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
mean_accuracy = scores.mean()
print("Mean Accuracy:", mean_accuracy)

Mean Accuracy: 0.7078795518207283


In [23]:
model.fit(X_train, y_train)
pred = model.predict(X_train)
training_accuracy= accuracy_score(y_train,pred)
print("Accuracy on Training data: ",training_accuracy)
pred = model.predict(X_test)
test_accuracy= accuracy_score(y_test,pred)
print("Accuracy on Test data: ",test_accuracy)
xrd_Bayes_crystal = label_encoder.inverse_transform(pred)
xrd_Bayes_crystal

Accuracy on Training data:  0.825503355704698
Accuracy on Test data:  0.6711409395973155


array(['cubic', 'triclinic', 'triclinic', 'triclinic', 'monoclinic',
       'triclinic', 'triclinic', 'cubic', 'triclinic', 'triclinic',
       'monoclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'monoclinic', 'triclinic',
       'monoclinic', 'triclinic', 'monoclinic', 'monoclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'monoclinic',
       'cubic', 'triclinic', 'triclinic', 'monoclinic', 'triclinic',
       'monoclinic', 'triclinic', 'monoclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'monoclinic', 'triclinic', 'triclinic',
       'monoclinic', 'triclinic', 'triclinic', 'cubic', 'triclinic',
       'triclinic', 'triclinic', 'orthorhombic', 'triclinic',
       'monoclinic', 'monoclinic', 'triclinic', 'monoclinic',
       'monoclinic', 'triclinic', 'monoclinic', 'triclinic', 'triclinic',
       'cubic', 'monoclinic', 'triclinic', 'monoclinic', 'monoclinic',
       'triclin

SCM descriptor

In [ ]:

df1= pd.read_csv('Sine_MP_train.csv')
df2= pd.read_csv('sine_mp_test.csv')
df=pd.concat([df1,df2],ignore_index=True)

In [12]:
X=df.iloc[:,14:]
Y=df['crystal_system']

In [13]:
from sklearn.preprocessing import StandardScaler
# create a scaler object
scaler = StandardScaler()

# fit scaler on the training data
scaler.fit(X)

# apply the scaler to both the training and test data
X_normalized = scaler.transform(X)

In [14]:
label_encoder = LabelEncoder()

# Fit and transform label encoder on target values
y_encoded = label_encoder.fit_transform(Y)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y_encoded, test_size=0.2, random_state=108)

In [16]:
# Define the search space for hyperparameters
param_space = {
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'gamma': hp.uniform('gamma', 0, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0, 5),
    'reg_lambda': hp.uniform('reg_lambda', 0, 5),
    'learning_rate': hp.loguniform('learning_rate', 0.01, 3),
    'n_estimators': hp.quniform('n_estimators', 10, 500, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1)
}

# Define the objective function to minimize (negative accuracy)
def objective(params):
    model = XGBClassifier(
        max_depth=int(params['max_depth']),
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        learning_rate=params['learning_rate'],
        n_estimators=int(params['n_estimators']),
        min_child_weight=int(params['min_child_weight'])
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return -accuracy  # Minimize negative accuracy

# Perform Bayesian optimization
best = fmin(
    fn=objective,
    space=param_space,
    algo=tpe.suggest,
    max_evals=50  # Number of optimization iterations
)

print("Best Hyperparameters:", best)

100%|██████████| 50/50 [02:20<00:00,  2.81s/trial, best loss: -0.697986577181208]
Best Hyperparameters: {'colsample_bytree': 0.7743065653585242, 'gamma': 0.8853771451686677, 'learning_rate': 2.002793313781907, 'max_depth': 10.0, 'min_child_weight': 8.0, 'n_estimators': 110.0, 'reg_alpha': 4.178615239137679, 'reg_lambda': 0.14349694310447236, 'subsample': 0.9917374956214849}


In [24]:
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, cross_val_score

# Define the best hyperparameters
best_hyperparameters = {
    'n_estimators': 110,
    'learning_rate': 2.002793313781907,
    'gamma': 0.8853771451686677,
    'max_depth': 10,
    'min_child_weight': 8,
    'colsample_bytree': 0.7743065653585242,
    'subsample': 0.9917374956214849,
    'reg_alpha': 4.178615239137679,
    'reg_lambda': 0.14349694310447236
}

# Create an XGBoost classifier with the best hyperparameters
model = XGBClassifier(**best_hyperparameters)

# Perform k-fold cross-validation
cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=1)
scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
mean_accuracy = scores.mean()
print("Mean Accuracy:", mean_accuracy)

Mean Accuracy: 0.6139019607843138


In [25]:
model.fit(X_train, y_train)
pred = model.predict(X_train)
training_accuracy= accuracy_score(y_train,pred)
print("Accuracy on Training data: ",training_accuracy)
pred = model.predict(X_test)
test_accuracy= accuracy_score(y_test,pred)
print("Accuracy on Test data: ",test_accuracy)
Sine_Bayes_crystal = label_encoder.inverse_transform(pred)
Sine_Bayes_crystal

Accuracy on Training data:  0.6325503355704698
Accuracy on Test data:  0.5771812080536913


array(['monoclinic', 'triclinic', 'trigonal', 'triclinic', 'monoclinic',
       'triclinic', 'triclinic', 'monoclinic', 'triclinic', 'triclinic',
       'monoclinic', 'triclinic', 'triclinic', 'monoclinic', 'triclinic',
       'monoclinic', 'cubic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'monoclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'monoclinic', 'triclinic', 'monoclinic',
       'monoclinic', 'monoclinic', 'triclinic', 'triclinic', 'monoclinic',
       'triclinic', 'triclinic', 'triclinic', 'monoclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'monoclinic', 'triclinic', 'triclinic', 'monoclinic', 'triclinic',
       'monoclinic', 'triclinic', 'cubic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'monoclinic', 'monoclinic',
       'triclinic', 'monoclinic', 'triclinic', 'monoclinic', 'triclinic',
       'triclinic', 'cubic', 'triclinic', 'triclinic'

OFM Descriptor

In [26]:
df1= pd.read_csv('orbital_MP_train.csv')
df2= pd.read_csv('Orbital_mp_test.csv')
df=pd.concat([df1,df2],ignore_index=True)

In [27]:
X=df.iloc[:,14:]
Y=df['crystal_system']

In [30]:
from sklearn.preprocessing import StandardScaler
# create a scaler object
scaler = StandardScaler()

# fit scaler on the training data
scaler.fit(X)

# apply the scaler to both the training and test data
X_normalized = scaler.transform(X)

In [31]:
label_encoder = LabelEncoder()

# Fit and transform label encoder on target values
y_encoded = label_encoder.fit_transform(Y)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y_encoded, test_size=0.2, random_state=108)

In [33]:
# Define the search space for hyperparameters
param_space = {
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'gamma': hp.uniform('gamma', 0, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0, 5),
    'reg_lambda': hp.uniform('reg_lambda', 0, 5),
    'learning_rate': hp.loguniform('learning_rate', 0.01, 3),
    'n_estimators': hp.quniform('n_estimators', 10, 500, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1)
}

# Define the objective function to minimize (negative accuracy)
def objective(params):
    model = XGBClassifier(
        max_depth=int(params['max_depth']),
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        learning_rate=params['learning_rate'],
        n_estimators=int(params['n_estimators']),
        min_child_weight=int(params['min_child_weight'])
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return -accuracy  # Minimize negative accuracy

# Perform Bayesian optimization
best = fmin(
    fn=objective,
    space=param_space,
    algo=tpe.suggest,
    max_evals=50  # Number of optimization iterations
)

print("Best Hyperparameters:", best)

100%|██████████| 50/50 [10:54<00:00, 13.09s/trial, best loss: -0.6577181208053692]
Best Hyperparameters: {'colsample_bytree': 0.9597365201988166, 'gamma': 2.544256659186939, 'learning_rate': 1.472842095003139, 'max_depth': 3.0, 'min_child_weight': 10.0, 'n_estimators': 150.0, 'reg_alpha': 2.728545659753868, 'reg_lambda': 0.5911161139003734, 'subsample': 0.7214186374252257}


In [34]:
import xgboost
from xgboost import XGBClassifier

# Define the best hyperparameters
best_hyperparameters = {
    'n_estimators': 150,
    'learning_rate': 1.472842095003139,
    'gamma': 2.544256659186939,
    'max_depth': 3,
    'min_child_weight': 10,
    'colsample_bytree': 0.9597365201988166,
    'subsample':0.7214186374252257,
    'reg_alpha': 2.728545659753868,
    'reg_lambda': 0.5911161139003734
}

# Create an XGBoost classifier with the best hyperparameters
model = XGBClassifier(**best_hyperparameters)

# Perform k-fold cross-validation
cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=1)
scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
mean_accuracy = scores.mean()
print("Mean Accuracy:", mean_accuracy)

Mean Accuracy: 0.6305882352941177


In [35]:
model.fit(X_train, y_train)
pred = model.predict(X_train)
training_accuracy= accuracy_score(y_train,pred)
print("Accuracy on Training data: ",training_accuracy)
pred = model.predict(X_test)
test_accuracy= accuracy_score(y_test,pred)
print("Accuracy on Test data: ",test_accuracy)
Orbital_Bayes_crystal = label_encoder.inverse_transform(pred)
Orbital_Bayes_crystal

Accuracy on Training data:  0.7075630252100841
Accuracy on Test data:  0.5906040268456376


array(['cubic', 'triclinic', 'monoclinic', 'triclinic', 'triclinic',
       'monoclinic', 'triclinic', 'monoclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'monoclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'monoclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'monoclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'monoclinic', 'triclinic', 'triclinic', 'triclinic', 'triclinic',
       'triclinic', 'triclinic', 'triclinic', 'monoclinic', 'monoclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclinic', 'monoclinic',
       'triclinic', 'monoclinic', 'triclinic', 'triclinic', 'monoclinic',
       'monoclinic', 'triclinic', 'triclinic', 'triclinic', 'monoclinic',
       'triclinic', 'triclinic', 'triclinic', 'triclin